In [1]:
import os
import pickle
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
from collections import defaultdict, Counter
from tqdm import tqdm

In [2]:
edges = pd.read_csv('edges_filtered.csv')

In [3]:
nodes = pd.read_csv("nodes_filtered.csv")

In [4]:
abv = pd.read_csv("abv.csv")
name_abv = dict(zip(abv.full_name, abv.abv))
abv_name = dict(zip(abv.abv, abv.full_name))

In [5]:
edges.head()

,START_ID,END_ID,TYPE,pmids,n_pmids,NEG,DOMAIN,PRED,RANGE
0,C0086931,C0037369,ABafAB,26656404;8816000;19745413;17710714;21412223;19...,6,False,AB,af,AB
1,C0950156,C0009671,ABafAB,12584106,1,False,AB,af,AB
2,C0024810,C0035651,ABafAB,15775958;11173168,2,False,AB,af,AB
3,C0556297,C0037369,ABafAB,12164677;9373700,2,False,AB,af,AB
4,C1154333,C0015745,ABafAB,23872405,1,False,AB,af,AB


In [6]:
nodes.head()

,ID,label,TYPE
0,C0038195,State Medicine,Activities & Behaviors
1,C1290952,Taking medication,Activities & Behaviors
2,C0680552,legalization,Activities & Behaviors
3,C0085092,Parenting behavior,Activities & Behaviors
4,C1096771,Murderer,Activities & Behaviors


In [7]:
nodes.TYPE.value_counts()

Chemicals & Drugs              63783
Living Beings                  47222
Disorders                      39876
protein                        29400
Genes & Molecular Sequences    20802
Anatomy                        15496
Procedures                     13781
Physiology                      8048
Concepts & Ideas                4265
Devices                         3873
genomic_entity                  1851
Objects                         1437
Activities & Behaviors          1386
Phenomena                       1159
Organizations                    892
Geographic Areas                 494
Occupations                      457
Name: TYPE, dtype: int64

In [8]:
# number of edges by node type
Counter(list(edges.DOMAIN) + list(edges.DOMAIN)).most_common()

[('CD', 14627962),
 ('G', 5869982),
 ('A', 5433270),
 ('DO', 2991932),
 ('PR', 1844272),
 ('LB', 1618726),
 ('PS', 731956),
 ('DV', 203056),
 ('PH', 104548),
 ('OR', 82326),
 ('AB', 80390),
 ('OB', 78912),
 ('CI', 67118),
 ('GA', 3998),
 ('OC', 2870)]

In [9]:
def print_most_common(node_type):
    x = edges[edges.DOMAIN == node_type].START_ID
    y = edges[edges.RANGE == node_type].END_ID
    c = [x[0] for x in Counter(list(x) + list(y)).most_common(10)]
    print(nodes.loc[c])

In [10]:
# [print_most_common(x) for x in set(edges.domain)]

In [11]:
# these are not useful or would be very difficult to normalize
remove_nodes = ['Concepts & Ideas', 'Organizations', 'Geographic Areas', 'Occupations', 'Devices',
               'Procedures']
remove_nodes = [name_abv[x] for x in remove_nodes]
remove_nodes

['CI', 'OR', 'GA', 'OC', 'DV', 'PR']

In [12]:
def drop_nodes(edges, remove_nodes):
    idx = edges[edges.DOMAIN.isin(remove_nodes) | edges.RANGE.isin(remove_nodes)].index
    edges.drop(idx, inplace=True)
print(len(edges))
drop_nodes(edges, remove_nodes)
print(len(edges))

16870659
15575145


In [13]:
nodes = nodes[nodes.ID.isin(edges.START_ID) | nodes.ID.isin(edges.END_ID)]

In [14]:
nodes.TYPE.value_counts()

Chemicals & Drugs              63674
Living Beings                  47099
Disorders                      38858
protein                        29386
Genes & Molecular Sequences    20801
Anatomy                        15289
Physiology                      8023
genomic_entity                  1848
Objects                         1437
Phenomena                       1148
Activities & Behaviors          1030
Name: TYPE, dtype: int64

In [15]:
nodes.to_csv("nodes_filtered2.csv", index=False)

In [16]:
edges.to_csv("edges_filtered2.csv", index=False)